In [ ]:
import torch.distributions
import transformer_lens
from IPython.display import Image
from transformer_lens import HookedTransformer

import core

%load_ext autoreload
%autoreload 2

In [ ]:
model = HookedTransformer.from_pretrained("gpt2-small")

In [ ]:
attn_only_4l = HookedTransformer.from_pretrained("attn-only-4l")

# IOI task

In [ ]:
threshold = 0.3
prompt = "When Mary and John went to the store, John gave a book to"
corrupt_prompt = "When Tom and Sarah went to the store, Felix gave a book to"
c = core.connectom(model, prompt,
                   core.logit_diff_metric(model, ' Mary', ' John'),
                   core.ZeroPattern(),
                   # core.CorruptIntervention(model, prompt, corrupt_prompt),
                   # core.CropIntervention(model, prompt),

                   core.BasicStrategy(),
                   # d.BacktrackBisectStrategy(threshold),
                   # d.BacktrackingStrategy(threshold),
                   # core.BisectStrategy(threshold),
                   # core.SplitStrategy(model, prompt, threshold),
                   )
core.plot_attn_connectome(model, prompt, c).show()
# graph = core.plot_graphviz_connectome(model, prompt, c, threshold=threshold).pipe('svg').decode('utf-8')
# SVG(graph)


In [ ]:
graph = core.plot_graphviz_connectome(model, prompt, c, threshold=0.09)

# Docstring task

In [ ]:
model = HookedTransformer.from_pretrained("attn-only-4l")
threshold = 0.3
prompt = '''def port(self, load, size, file, last):
    """oil column piece

    :param load: crime population
    :param size: unit dark
    :param'''
corrupt_prompt = prompt.replace("load", "banana").replace("size", "apple").replace("file", "pear").replace("last", "orange")

c = core.connectom(model, prompt,
                   core.logit_diff_metric(model, ' file', ' self', ' load', ' size', ' last'),
                   core.ZeroPattern(),
                   # core.CorruptIntervention(model, prompt, corrupt_prompt),

                   # core.BasicStrategy(),
                   core.SplitStrategy(model, prompt, threshold),
                   # core.BacktrackBisectStrategy(threshold),
                   # d.BacktrackingStrategy(threshold),
                   # core.BisectStrategy(threshold),
                   )

In [ ]:
graph_threshold = 0.6
core.plot_attn_connectome(model, prompt, c).show()
# graph = core.plot_graphviz_connectome(model, prompt, c, graph_threshold, depth=2).pipe('svg').decode('utf-8')
graph = core.plot_graphviz_connectome(model, prompt, c, graph_threshold, depth=2).pipe('png')
Image(graph)
# SVG(graph)


In [ ]:
print(prompt)
transformer_lens.utils.test_prompt(prompt, ' file', model)

# Code task on Pythia

In [ ]:
model = HookedTransformer.from_pretrained('gpt2-medium')

In [ ]:
prompt = """from typing import List, Dict
def f(x: List):
    return sum(x[::2])

def g(x: float):
    return x ** 2

def h(x: Dict):
    return sum(x.values())

def i(x: str):
    return len(x)

var1: str = 'abc'
var2: Dict = {'a': 1, 'b': 2}
var3: List = [1, 2, 3]
var4: int = 4

h(var"""

transformer_lens.utils.test_prompt(prompt, '1', model)


# Exploration of grouping techniques

In [ ]:
# Finding the log-probs on the prompt tokens
prompt = "When Mary and John went to the store, John gave a book to Mary."
log_probs = model(prompt)[0].log_softmax(-1)  # (seq_len, vocab_size)
tokens = model.to_tokens(prompt)[0]
tokens_str = model.to_str_tokens(tokens)
print(tokens.shape)
print(log_probs.shape)
correct_logprobs = log_probs[torch.arange(len(tokens) - 1), tokens[1:]]
print(correct_logprobs.shape)
for i, (t, n, l) in enumerate(zip(tokens_str, tokens_str[1:], correct_logprobs)):
    print(f"{i:2d} {t!r} {l.item():.2f} -> {n!r}")

import plotly.express as px
px.line(x=[f'{i} {t!r} -> {n!r}' for i, (t, n) in enumerate(zip(tokens_str, tokens_str[1:]))],
        y=correct_logprobs.detach() * 0)

In [ ]:
prompt = '''def port(self, load, size, file, last):
    """oil column piece

    :param load: crime population
    :param size: unit dark
    :param'''

print(prompt)
s = core.SplitStrategy(model, prompt, 0.1,
                       (
                           '\n\n',
                           tuple('.!?'),
                           tuple(',:;'),
                       )
                       )
s.show_tree()

In [ ]:
len(core.filter_connectome(c, 1))

In [ ]:
len(c)